In [20]:
import numpy as np
import scipy.stats as sts
import scipy.special as spec
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integral
import pandas as pd
import numpy.linalg as lin
%matplotlib notebook

dt = pd.read_csv('NewMacroSeries.txt', header = None)
dt.columns = ['c','k','w','r','y']

In [21]:
unif_vals_1 = sts.uniform.rvs(0, 1, size=(100,1000))

In [22]:
DT_C = np.array(dt.c.values)
DT_K = np.array(dt.k.values)
DT_W = np.array(dt.w.values)
DT_R = np.array(dt.r.values)
DT_Y = np.array(dt.y.values)

K_1 = DT_K.mean()

In [23]:
#Generate a Matrix from uniform distribution.
def norm_draws(unif_vals, mu, sigma):
    norm_draws = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    
    return norm_draws

In [24]:
#Get the value of z
def get_z(rho, unif_vals, mu, sigma):
    Z = np.ones((100,1000))
    Z = np.array(Z)
    norm_draw = norm_draws(unif_vals, mu, sigma)
    norm_draw = np.array(norm_draw)
    Z[0,:] = mu + norm_draw[0,:]
    for i in range(99):
        Z[i+1,:] = np.multiply(rho,Z[i,:]) + (1-rho)*mu + norm_draw[i+1,:]
    
    return Z

In [25]:
#Get simulated value for k
def get_simulated_k(alpha, rho, unif_vals, mu, sigma):
    Z = get_z(rho, unif_vals, mu, sigma)
    K = np.ones((101,1000))
    K = np.array(K)
    K[0,:] = K_1
    for i in range(100):
        K[i+1,:] = np.multiply(alpha*0.99, np.multiply(np.exp(Z[i,:]), (K[i,:])**alpha))
    
    return K

In [26]:
#Get simulated w
def get_simulated_w(alpha, rho, unif_vals, mu, sigma):
    Z = get_z(rho, unif_vals, mu, sigma)
    K = get_simulated_k(alpha, rho, unif_vals, mu, sigma)
    W = np.ones((100,1000))
    W = np.array(W)
    for i in range(100):
        W[i,:] = np.multiply(np.exp(Z[i,:]), (K[i,:])**alpha) * (1-alpha)
    #np.multiply((1 - alpha), np.multiply(np.exp(Z[i,:]), (K[i,:])**alpha))
    return W

In [27]:
#Get simulated r
def get_simulated_r(alpha, rho, unif_vals, mu, sigma):
    Z = get_z(rho, unif_vals, mu, sigma)
    K = get_simulated_k(alpha, rho, unif_vals, mu, sigma)
    R = np.ones((100,1000))
    R = np.array(R)
    for i in range(100):
        R[i,:] = np.multiply(alpha, np.multiply(np.exp(Z[i,:]), (K[i,:])**(alpha - 1)))
    
    return R

In [28]:
#Get simulated c
def get_simulated_c(alpha, rho, unif_vals, mu, sigma):
    K = get_simulated_k(alpha, rho, unif_vals, mu, sigma)
    W = get_simulated_w(alpha, rho, unif_vals, mu, sigma)
    R = get_simulated_r(alpha, rho, unif_vals, mu, sigma)
    C = np.ones((100,1000))
    for i in range(100):
        C[i,:] = W[i,:] + np.multiply(R[i,:], K[i,:]) - K[i+1,:]
        
    return C

In [29]:
#Get simelated y
def get_simulated_y(alpha, rho, unif_vals, mu, sigma):
    Z = get_z(rho, unif_vals, mu, sigma)
    K = get_simulated_k(alpha, rho, unif_vals, mu, sigma)
    Y = np.ones((100,1000))
    for i in range(100):
        Y[i,:] = np.multiply(np.exp(Z[i,:]), (K[i,:])**alpha)
        
    return Y

In [30]:
#Get data moment
def model_moments(alpha, rho, unif_vals, mu, sigma):
    Z = get_z(rho, unif_vals, mu, sigma)
    K = get_simulated_k(alpha, rho, unif_vals, mu, sigma)
    W = get_simulated_w(alpha, rho, unif_vals, mu, sigma)
    R = get_simulated_r(alpha, rho, unif_vals, mu, sigma)
    C = get_simulated_c(alpha, rho, unif_vals, mu, sigma)
    Y = get_simulated_y(alpha, rho, unif_vals, mu, sigma)
    
    #Get the simulated moment for mean(C)
    meanC_simu = np.ones(1000)
    for i in range(1000):
        meanC_simu[i] = C[:,i].mean()
        
    mmC = meanC_simu.mean()
    
    #Get the simulated moment for mean(K)
    meanK_simu = np.ones(1000)
    for i in range(1000):
        meanK_simu[i] = K[:100,i].mean()
        
    mmK = meanK_simu.mean()
    
    #Get the simulated moment for mean(C/Y)
    ConY_simu = np.ones((100,1000))
    ConY_simu = np.multiply(C,Y**(-1))
    meanConY_simu = np.ones(1000)
    for i in range(1000):
        meanConY_simu[i] = ConY_simu[:,i].mean()
        
    mmConY = meanConY_simu.mean()
    
    #Get the simulated moment for var(Y)
    varY_simu = np.ones(1000)
    for i in range(1000):
        varY_simu[i] = Y[:,i].std()
        
    mmY = varY_simu.mean()
    
    #Get the simulated moment for corr(Ct,Ct-1)
    corrCC_simu = np.ones(1000)
    for i in range(1000):
        a = np.ones(99)
        b = np.ones(99)
        a = C[:99,i]
        b = C[1:100,i]
        corrCC_simu[i] = sts.pearsonr(a, b)[0]
        
    mmCC = corrCC_simu.mean()
    
    
    #Get the simulated moment for corr(Kt,Ct)
    corrCK_simu = np.ones(1000)
    for i in range(1000):
        corrCK_simu[i] = sts.pearsonr(C[:,i], K[:100,i])[0]
        
    mmCK = corrCK_simu.mean()
    
    
    return np.array([mmC,mmK,mmConY,mmY,mmCC,mmCK])

In [31]:
model_moments(0.4, 0.9, unif_vals_1, 10, 0.1)

array([  6.04375675e+06,   3.98991285e+06,   6.04000000e-01,
         3.18129684e+06,   9.27704770e-01,   9.16349243e-01])

In [32]:
#Get the data moment
dm_mC = DT_C.mean()
dm_mK = DT_K.mean()
dm_mCY = np.multiply(DT_C, DT_Y**(-1)).mean()
dm_vY = DT_Y.std()
dm_CC = sts.pearsonr(DT_C[1:], DT_C[:-1])[0]
dm_CK = sts.pearsonr(DT_C, DT_K)[0]

In [33]:
#Compute the error vector
def err_vec(alpha, rho, unif_vals, mu, sigma):
    mom_model = model_moments(alpha, rho, unif_vals, mu, sigma)
    
    mom_data = np.array([dm_mC, dm_mK, dm_mCY, dm_vY, dm_CC, dm_CK])
    
    err = (mom_model - mom_data) / mom_data
    
    return err

In [34]:
err_vec(0.4, 0.9, unif_vals_1, 10, 0.1)

array([-0.34885874, -0.39946993,  0.0338925 , -0.40280709, -0.01366678,
       -0.02599249])

In [35]:
#Compute criterion function
def crit(params, *args):
    alpha, rho, mu, sigma = params
    unif_vals, W_hat = args
    
    err = err_vec(alpha, rho, unif_vals, mu, sigma)
    
    crit_val = np.dot(np.dot(err.T, W_hat), err)
    
    return crit_val

In [36]:
alpha_init = 0.4
rho_init = 0.9
mu_init = 11.5
sigma_init = 0.1
W_hat = np.eye(6)

params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
smm_args = (unif_vals_1, W_hat)
results = opt.minimize(crit, params_init, args=(smm_args),
                          method='L-BFGS-B',
                          bounds=((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1)))

alpha_smm, rho_smm, mu_smm, sigma_smm = results.x
print (' alpha_smm=', alpha_smm, ' rho_smm=', rho_smm, ' mu_smm=', mu_smm, ' sigma_smm=', sigma_smm, \
      ' err_vec=', err_vec(alpha_smm, rho_smm, unif_vals_1, mu_smm, sigma_smm), ' crit=', \
      np.dot(np.dot(err_vec(alpha_smm, rho_smm, unif_vals_1, mu_smm, sigma_smm).T, W_hat), \
      err_vec(alpha_smm, rho_smm, unif_vals_1, mu_smm, sigma_smm)))

 alpha_smm= 0.421015095587  rho_smm= 0.921055757104  mu_smm= 9.92499026117  sigma_smm= 0.0935627523776  err_vec= [  7.18163236e-04  -7.25699947e-04  -1.72020649e-03  -2.73511101e-07
   2.98001516e-04  -2.97233448e-04]  crit= 4.17866191398e-06
